Adult income dataset http://archive.ics.uci.edu/ml/datasets/Adult <br>
Data https://www.kaggle.com/uciml/adult-census-income or 'data/adult.csv'<br>
Missing attribute values denoted by "?"

In [0]:
import pandas as pd
import numpy as np

adults = pd.read_csv('data/adult.csv')


In [0]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(adults.drop('income', 1), adults.income, random_state=2, test_size=0.3, stratify=adults.income)

print(Xtrain.shape, Xtest.shape)


(22792, 14) (9769, 14)


In [0]:
ytrain.head(3)

31249     >50K
19086    <=50K
10615     >50K
Name: income, dtype: object

In [0]:
Xtrain.head(3)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
31249,52,Local-gov,153312,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
19086,19,Private,219300,Some-college,10,Never-married,Sales,Own-child,White,Female,0,0,25,United-States
10615,44,Self-emp-not-inc,197558,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,70,United-States


In [0]:
adults.head(3)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K


In [0]:
from sklearn import preprocessing
factors_Xtrain = Xtrain[['workclass','education','marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']]
factors_Xtest = Xtest[['workclass','education','marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']]


enc = preprocessing.OneHotEncoder()
enc.fit(factors_Xtrain)
enc.categories_

[array(['?', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private',
        'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'],
       dtype=object),
 array(['10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th',
        'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad',
        'Masters', 'Preschool', 'Prof-school', 'Some-college'],
       dtype=object),
 array(['Divorced', 'Married-AF-spouse', 'Married-civ-spouse',
        'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed'],
       dtype=object),
 array(['?', 'Adm-clerical', 'Armed-Forces', 'Craft-repair',
        'Exec-managerial', 'Farming-fishing', 'Handlers-cleaners',
        'Machine-op-inspct', 'Other-service', 'Priv-house-serv',
        'Prof-specialty', 'Protective-serv', 'Sales', 'Tech-support',
        'Transport-moving'], dtype=object),
 array(['Husband', 'Not-in-family', 'Other-relative', 'Own-child',
        'Unmarried', 'Wife'], dtype=object),
 array(['Amer-Indian-Eskimo'

In [0]:
transform_Xtrain= enc.transform(factors_Xtrain).toarray()
transform_Xtest= enc.transform(factors_Xtest).toarray()
print(transform_Xtrain)
print(transform_Xtest)


[[0. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


In [0]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(random_state = 2)

gbc.fit(transform_Xtrain,ytrain)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=2, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [0]:
# Accuracy on train

gbc.score(transform_Xtrain, ytrain)

0.8342400842400842

In [0]:
# Accuracy on test
gbc.score(transform_Xtest, ytest)

In [0]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report

ypred_test = gbc.predict(transform_Xtest)
print(confusion_matrix(ytest, ypred_test))

[[6860  557]
 [1105 1247]]


In [0]:
print(classification_report(ytest, ypred_test))

              precision    recall  f1-score   support

       <=50K       0.86      0.92      0.89      7417
        >50K       0.69      0.53      0.60      2352

    accuracy                           0.83      9769
   macro avg       0.78      0.73      0.75      9769
weighted avg       0.82      0.83      0.82      9769

